In [1]:
import numpy as np

import pandas as pd

import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping





def prepare_data(path,target_column_start,remove_threshold = 10):

    



    df = pd.read_csv(path, index_col=0)

    

    # delete all feature columns that have less than remove_threshold number of entries

    for col in df.columns:

        if df[col].astype(bool).sum() < remove_threshold:

            df.drop(col,inplace=True,axis=1)

            

    # get all target columns

    target_columns = list(df.columns[list(df.columns).index(target_column_start):])

            

    # get column indices

    col_indices = {col: i for i, col in enumerate(df.columns)}

    target_indices = [col_indices[i] for i in target_columns] 

    

    # split data into train, validation and test

    data_len = len(df)

    train_data = df[0:int(data_len*0.7)]

    val_data = df[int(data_len*0.7):int(data_len*0.9)]

    test_data = df[int(data_len*0.9):]

    num_features = len(target_columns)

    

    return df,target_columns, target_indices,train_data,val_data,test_data, num_features





class create_lookback():

    

    # create lookback class that gives the deep learning model size_input number of elements to predict next eleement from

    

    def __init__(self, size_input,train_data, val_data, test_data,cols=None):



        # initialization of all variables

        self.train_data = train_data

        self.val_data = val_data

        self.test_data = test_data

        self.cols = cols

        

        if cols is not None:

          self.cols_indices = {name: i for i, name in enumerate(cols)}

        self.column_indices = {name: i for i, name in enumerate(train_data.columns)}

        

        self.size_input = size_input

        self.size_lookback = size_input + 1

        self.input_slice = slice(0, size_input)

        self.input_idx = np.arange(self.size_lookback)[self.input_slice]

        self.label_start = self.size_lookback - 1

        self.labels_slice = slice(self.label_start, None)

        self.label_idx = np.arange(self.size_lookback)[self.labels_slice]





    def split(self, features):

        

        # split and reshape dataset  to correct inputs and labels for deep learnin model

        

        inputs = features[:, self.input_slice, :]

        labels = features[:, self.labels_slice, :]

        

        if self.cols is not None:

            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.cols],axis=-1)

    

        inputs.set_shape([None, self.size_input, None])

        labels.set_shape([None, 1, None])



        return inputs, labels







    def create_keras_data(self, data):

        

        # create final data with size_lookback elements, stride movement of 1 and batch_size 16 

        

        data = np.array(data, dtype=np.float32)

        keras_data = tf.keras.preprocessing.timeseries_dataset_from_array( data=data, targets=None, sequence_length=self.size_lookback,

                sequence_stride=1, shuffle=True, batch_size=16,)

        keras_data = keras_data.map(self.split)

        return keras_data



    

    @property

    def train(self):

        return self.create_keras_data(self.train_data)

    

    @property

    def val(self):

        return self.create_keras_data(self.val_data)

    

    @property

    def test(self):

        return self.create_keras_data(self.test_data)







class Baseline(tf.keras.Model):

    #create baseline class that returns back values from previous steps

  def __init__(self, label_idx= [], label_max_count = 9999):

    super().__init__()

    self.label_idx = label_idx

    self.label_max_count = label_max_count



  

  def call(self, inputs):

    if len(self.label_idx) == 0:

      return inputs

    if len(self.label_idx) > 0 and len(self.label_idx)<self.label_max_count:

        result = inputs[:,:, self.label_idx[0]-1:self.label_idx[-1]]

        return result

    result = inputs[:, :, self.label_idx]

    return result[:, :, tf.newaxis]







def model_compile(model, window, patience=10,max_epochs=100):

    # define early stopping

    stop_fcn = EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    # compile deep learning model 

    model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics= [tf.metrics.MeanSquaredError(), tf.metrics.MeanAbsoluteError()])

    # train deep learning model with predefined max epoch, and stopping function 

    out = model.fit(window.train, epochs = max_epochs, validation_data=window.val, callbacks= stop_fcn)

    return model,out





def accuracy_fcn(Y_predicted, Y_groundtruth, threshold = 0.1):



    # threshold  determines which values are considered small enough to be  considered as "not attacked" targets

    # accuracy function for elementwise and setwise accuracy for targer prediction



    cnt = 0

    nonzero_cnt = 0

    cnt_setwise = 0

    nonzero_setwise = 0



    Y_predicted = [[0 if i < threshold else i for i in j] for j in Y_predicted]



    for i in range(len(Y_predicted)):



        Y_pred_curr = Y_predicted[i]

        Y_gt_curr = Y_groundtruth[i]

        

        correct_set = {}

        

        # logic to account for no attack days and to handle target attacked days

                

        if np.count_nonzero(Y_gt_curr) == 0:

            if np.count_nonzero(Y_pred_curr)==0:

                correct_set = [1,1]

                idx_predicted = [None,None]

                idx_groundtruth = [None,None]

            

        elif np.count_nonzero(Y_gt_curr) == 1:

            idx_predicted = np.array(Y_pred_curr).argsort()[-1:][::-1]

            idx_groundtruth = np.array(Y_gt_curr).argsort()[-1:][::-1]

            if np.count_nonzero(Y_pred_curr) == 1:

                idx_predicted = [list(idx_predicted)[0],99]

                idx_groundtruth = [list(idx_groundtruth)[0],99]

            if np.count_nonzero(Y_pred_curr) == 0:

                idx_predicted = [99,99]

                idx_groundtruth = [list(idx_groundtruth)[0],99]

            else:

                idx_predicted = np.array(Y_pred_curr).argsort()[-2:][::-1]

                idx_groundtruth = [list(idx_groundtruth)[0],99]

            correct_set = set(idx_groundtruth).intersection(set(idx_predicted))



        # otherwise just compare top 2 entries 

            

        else:

            idx_predicted = np.array(Y_pred_curr).argsort()[-2:][::-1]

            idx_groundtruth = np.array(Y_gt_curr).argsort()[-2:][::-1]



            correct_set = set(idx_groundtruth).intersection(set(idx_predicted))

            

        if len(correct_set) >= 1:



            cnt_setwise += 1;



        nonzero_setwise += 1;

        cnt += len(correct_set)

        nonzero_cnt += 2



    print('| setwise accuracy = %f' % (cnt / nonzero_cnt))

    print('| eventwise accuracy = %f' % (cnt_setwise / nonzero_setwise))

    return float(cnt / nonzero_cnt),float(cnt_setwise / nonzero_setwise)





def model_eval(model,frame):

    

    # evalation over test data for elementiwse and setwise accuracy 

    predicted_vals = []

    groundtruth_vals = []

    for test_X,test_Y in frame.test:

        out = model(test_X).numpy()

        for i in range(test_Y.shape[0]):

            predicted_vals.append(out[i][0])

            groundtruth_vals.append(test_Y[i][0])

    accuracy_fcn(predicted_vals,groundtruth_vals)

In [2]:
import tensorflow as tf
import random
#from support_functions import model_eval, create_lookback, prepare_data, Baseline
import warnings
import os
warnings.filterwarnings("ignore")

## Baseline

In [3]:
# random seed set for reproducability
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(5)
random.seed(1254)
tf.random.set_seed(5)

In [4]:
''' import dataset: the script is based on the Afghanistan dataset,
however, one can perform the models using the Iraq one. 
When using the Iraq dataset, target_column_start 
should be as well: 'Private Citizens & Property' '''

path = 'C:\\Users\\Gian Maria\\Desktop\\AAAI_21\\december\\afg_shallow.csv' # alternatively: 'iraq_time_series01.csv'
target_column_start =  'Airports & Aircraft'

df,target_columns, target_indices,train_df,val_df,test_df, num_features = prepare_data(path,target_column_start)


# define the lookback 
loockback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)


baseline = Baseline(target_indices,len(df.columns))
model_eval(baseline,loockback_frame)

| setwise accuracy = 0.157012
| eventwise accuracy = 0.314024


## Dense 

### Dense Lookback 1

In [5]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame)
model_eval(dense,lookback_frame)

Epoch 1/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0550 - mean_squared_error: 0.0550 - mean_absolute_error: 0.1128 - val_loss: 0.0601 - val_mean_squared_error: 0.0601 - val_mean_absolute_error: 0.1566
Epoch 2/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0414 - mean_squared_error: 0.0414 - mean_absolute_error: 0.0973 - val_loss: 0.0502 - val_mean_squared_error: 0.0502 - val_mean_absolute_error: 0.1362
Epoch 3/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0398 - mean_squared_error: 0.0398 - mean_absolute_error: 0.0933 - val_loss: 0.0476 - val_mean_squared_error: 0.0476 - val_mean_absolute_error: 0.1305
Epoch 4/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0392 - mean_squared_error: 0.0392 - mean_absolute_error: 0.0907 - val_loss: 0.0470 - val_mean_squared_error: 0.0470 - val_mean_absolute_error: 0.1279
Epoch 5/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0387 - mean_squ

### Dense Lookback 5

In [6]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame5)
model_eval(dense,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 2s 10ms/step - loss: 0.0552 - mean_squared_error: 0.0552 - mean_absolute_error: 0.1194 - val_loss: 0.0688 - val_mean_squared_error: 0.0688 - val_mean_absolute_error: 0.1714
Epoch 2/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0407 - mean_squared_error: 0.0407 - mean_absolute_error: 0.0977 - val_loss: 0.0545 - val_mean_squared_error: 0.0545 - val_mean_absolute_error: 0.1480
Epoch 3/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0386 - mean_squared_error: 0.0386 - mean_absolute_error: 0.0927 - val_loss: 0.0486 - val_mean_squared_error: 0.0486 - val_mean_absolute_error: 0.1337
Epoch 4/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0377 - mean_squared_error: 0.0377 - mean_absolute_error: 0.0895 - val_loss: 0.0464 - val_mean_squared_error: 0.0464 - val_mean_absolute_error: 0.1288
Epoch 5/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0372 - mean_sq

### Dense Lookback 15

In [7]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame15)
model_eval(dense,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 1s 7ms/step - loss: 0.0538 - mean_squared_error: 0.0538 - mean_absolute_error: 0.1132 - val_loss: 0.0554 - val_mean_squared_error: 0.0554 - val_mean_absolute_error: 0.1498
Epoch 2/100
143/143 [==============================] - 1s 4ms/step - loss: 0.0400 - mean_squared_error: 0.0400 - mean_absolute_error: 0.0929 - val_loss: 0.0437 - val_mean_squared_error: 0.0437 - val_mean_absolute_error: 0.1227
Epoch 3/100
143/143 [==============================] - 1s 4ms/step - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.0886 - val_loss: 0.0431 - val_mean_squared_error: 0.0431 - val_mean_absolute_error: 0.1236
Epoch 4/100
143/143 [==============================] - 1s 5ms/step - loss: 0.0375 - mean_squared_error: 0.0375 - mean_absolute_error: 0.0873 - val_loss: 0.0425 - val_mean_squared_error: 0.0425 - val_mean_absolute_error: 0.1212
Epoch 5/100
143/143 [==============================] - 1s 6ms/step - loss: 0.0369 - mean_squ

### Dense Lookback 30

In [8]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame30)
model_eval(dense,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 1s 10ms/step - loss: 0.0524 - mean_squared_error: 0.0524 - mean_absolute_error: 0.1108 - val_loss: 0.0523 - val_mean_squared_error: 0.0523 - val_mean_absolute_error: 0.1498
Epoch 2/100
142/142 [==============================] - 1s 4ms/step - loss: 0.0393 - mean_squared_error: 0.0393 - mean_absolute_error: 0.0923 - val_loss: 0.0444 - val_mean_squared_error: 0.0444 - val_mean_absolute_error: 0.1235
Epoch 3/100
142/142 [==============================] - 1s 5ms/step - loss: 0.0376 - mean_squared_error: 0.0376 - mean_absolute_error: 0.0876 - val_loss: 0.0414 - val_mean_squared_error: 0.0414 - val_mean_absolute_error: 0.1120
Epoch 4/100
142/142 [==============================] - 1s 4ms/step - loss: 0.0368 - mean_squared_error: 0.0368 - mean_absolute_error: 0.0859 - val_loss: 0.0407 - val_mean_squared_error: 0.0407 - val_mean_absolute_error: 0.1100
Epoch 5/100
142/142 [==============================] - 1s 5ms/step - loss: 0.0361 - mean_sq

## LSTM

### LSTM Lookback 1

In [9]:
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame)
model_eval(lstm,lookback_frame)

Epoch 1/100
144/144 [==============================] - 2s 15ms/step - loss: 0.0449 - mean_squared_error: 0.0449 - mean_absolute_error: 0.0963 - val_loss: 0.0451 - val_mean_squared_error: 0.0451 - val_mean_absolute_error: 0.1201
Epoch 2/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0397 - mean_squared_error: 0.0397 - mean_absolute_error: 0.0921 - val_loss: 0.0411 - val_mean_squared_error: 0.0411 - val_mean_absolute_error: 0.1148
Epoch 3/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0393 - mean_squared_error: 0.0393 - mean_absolute_error: 0.0899 - val_loss: 0.0417 - val_mean_squared_error: 0.0417 - val_mean_absolute_error: 0.1118
Epoch 4/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0391 - mean_squared_error: 0.0391 - mean_absolute_error: 0.0884 - val_loss: 0.0409 - val_mean_squared_error: 0.0409 - val_mean_absolute_error: 0.1094
Epoch 5/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0388 - mean_sq

### LSTM Lookback 5

In [10]:
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame5)
model_eval(lstm,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 2s 16ms/step - loss: 0.0459 - mean_squared_error: 0.0459 - mean_absolute_error: 0.1069 - val_loss: 0.0425 - val_mean_squared_error: 0.0425 - val_mean_absolute_error: 0.1213
Epoch 2/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0399 - mean_squared_error: 0.0399 - mean_absolute_error: 0.0933 - val_loss: 0.0407 - val_mean_squared_error: 0.0407 - val_mean_absolute_error: 0.1146
Epoch 3/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0391 - mean_squared_error: 0.0391 - mean_absolute_error: 0.0896 - val_loss: 0.0397 - val_mean_squared_error: 0.0397 - val_mean_absolute_error: 0.1116
Epoch 4/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0387 - mean_squared_error: 0.0387 - mean_absolute_error: 0.0882 - val_loss: 0.0401 - val_mean_squared_error: 0.0401 - val_mean_absolute_error: 0.1108
Epoch 5/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0385 - mean_sq

#### LSTM Lookback 15

In [11]:
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame15)
model_eval(lstm,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 4s 28ms/step - loss: 0.0456 - mean_squared_error: 0.0456 - mean_absolute_error: 0.1093 - val_loss: 0.0420 - val_mean_squared_error: 0.0420 - val_mean_absolute_error: 0.1208
Epoch 2/100
143/143 [==============================] - 2s 14ms/step - loss: 0.0394 - mean_squared_error: 0.0394 - mean_absolute_error: 0.0919 - val_loss: 0.0400 - val_mean_squared_error: 0.0400 - val_mean_absolute_error: 0.1133
Epoch 3/100
143/143 [==============================] - 2s 14ms/step - loss: 0.0386 - mean_squared_error: 0.0386 - mean_absolute_error: 0.0883 - val_loss: 0.0395 - val_mean_squared_error: 0.0395 - val_mean_absolute_error: 0.1097
Epoch 4/100
143/143 [==============================] - 2s 12ms/step - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.0868 - val_loss: 0.0388 - val_mean_squared_error: 0.0388 - val_mean_absolute_error: 0.1088
Epoch 5/100
143/143 [==============================] - 2s 13ms/step - loss: 0.0380 - mea

Epoch 37/100
143/143 [==============================] - 2s 14ms/step - loss: 0.0370 - mean_squared_error: 0.0370 - mean_absolute_error: 0.0821 - val_loss: 0.0371 - val_mean_squared_error: 0.0371 - val_mean_absolute_error: 0.0995
Epoch 38/100
143/143 [==============================] - 2s 16ms/step - loss: 0.0369 - mean_squared_error: 0.0369 - mean_absolute_error: 0.0821 - val_loss: 0.0371 - val_mean_squared_error: 0.0371 - val_mean_absolute_error: 0.0998
Epoch 39/100
143/143 [==============================] - 2s 17ms/step - loss: 0.0368 - mean_squared_error: 0.0368 - mean_absolute_error: 0.0819 - val_loss: 0.0369 - val_mean_squared_error: 0.0369 - val_mean_absolute_error: 0.1004
Epoch 40/100
143/143 [==============================] - 2s 17ms/step - loss: 0.0369 - mean_squared_error: 0.0369 - mean_absolute_error: 0.0819 - val_loss: 0.0370 - val_mean_squared_error: 0.0370 - val_mean_absolute_error: 0.0996
Epoch 41/100
143/143 [==============================] - 2s 15ms/step - loss: 0.0369 

### LSTM Lookback 30

In [12]:
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame30)
model_eval(lstm,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 8s 59ms/step - loss: 0.0445 - mean_squared_error: 0.0445 - mean_absolute_error: 0.1064 - val_loss: 0.0422 - val_mean_squared_error: 0.0422 - val_mean_absolute_error: 0.1225
Epoch 2/100
142/142 [==============================] - 4s 29ms/step - loss: 0.0391 - mean_squared_error: 0.0391 - mean_absolute_error: 0.0915 - val_loss: 0.0412 - val_mean_squared_error: 0.0412 - val_mean_absolute_error: 0.1180
Epoch 3/100
142/142 [==============================] - 4s 30ms/step - loss: 0.0384 - mean_squared_error: 0.0384 - mean_absolute_error: 0.0881 - val_loss: 0.0410 - val_mean_squared_error: 0.0410 - val_mean_absolute_error: 0.1149
Epoch 4/100
142/142 [==============================] - 4s 29ms/step - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.0863 - val_loss: 0.0415 - val_mean_squared_error: 0.0415 - val_mean_absolute_error: 0.1187
Epoch 5/100
142/142 [==============================] - 3s 24ms/step - loss: 0.0378 - mea

## CNN

### Lookback 1

In [13]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)


conv_width = 1
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame)
model_eval(conv,lookback_frame)

Epoch 1/100
144/144 [==============================] - 3s 21ms/step - loss: 0.0999 - mean_squared_error: 0.0999 - mean_absolute_error: 0.1470 - val_loss: 0.1286 - val_mean_squared_error: 0.1286 - val_mean_absolute_error: 0.2418
Epoch 2/100
144/144 [==============================] - 2s 13ms/step - loss: 0.0510 - mean_squared_error: 0.0510 - mean_absolute_error: 0.1141 - val_loss: 0.0763 - val_mean_squared_error: 0.0763 - val_mean_absolute_error: 0.1786
Epoch 3/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0433 - mean_squared_error: 0.0433 - mean_absolute_error: 0.1037 - val_loss: 0.0650 - val_mean_squared_error: 0.0650 - val_mean_absolute_error: 0.1607
Epoch 4/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0408 - mean_squared_error: 0.0408 - mean_absolute_error: 0.0991 - val_loss: 0.0596 - val_mean_squared_error: 0.0596 - val_mean_absolute_error: 0.1521
Epoch 5/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0398 - mean

### Lookback 5

In [14]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)


conv_width = 5
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame5)
model_eval(conv,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0568 - mean_squared_error: 0.0568 - mean_absolute_error: 0.1162 - val_loss: 0.0687 - val_mean_squared_error: 0.0687 - val_mean_absolute_error: 0.1718
Epoch 2/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0412 - mean_squared_error: 0.0412 - mean_absolute_error: 0.0979 - val_loss: 0.0521 - val_mean_squared_error: 0.0521 - val_mean_absolute_error: 0.1447
Epoch 3/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0387 - mean_squared_error: 0.0387 - mean_absolute_error: 0.0929 - val_loss: 0.0473 - val_mean_squared_error: 0.0473 - val_mean_absolute_error: 0.1345
Epoch 4/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0377 - mean_squared_error: 0.0377 - mean_absolute_error: 0.0900 - val_loss: 0.0464 - val_mean_squared_error: 0.0464 - val_mean_absolute_error: 0.1302
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0371 - mean_s

### LSTM Lookback 15

In [15]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)


conv_width = 15
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame15)
model_eval(conv,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 2s 12ms/step - loss: 0.0510 - mean_squared_error: 0.0510 - mean_absolute_error: 0.1082 - val_loss: 0.0488 - val_mean_squared_error: 0.0488 - val_mean_absolute_error: 0.1431
Epoch 2/100
143/143 [==============================] - 1s 6ms/step - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.0915 - val_loss: 0.0442 - val_mean_squared_error: 0.0442 - val_mean_absolute_error: 0.1299
Epoch 3/100
143/143 [==============================] - 1s 5ms/step - loss: 0.0375 - mean_squared_error: 0.0375 - mean_absolute_error: 0.0874 - val_loss: 0.0419 - val_mean_squared_error: 0.0419 - val_mean_absolute_error: 0.1215
Epoch 4/100
143/143 [==============================] - 1s 7ms/step - loss: 0.0368 - mean_squared_error: 0.0368 - mean_absolute_error: 0.0866 - val_loss: 0.0419 - val_mean_squared_error: 0.0419 - val_mean_absolute_error: 0.1210
Epoch 5/100
143/143 [==============================] - 1s 6ms/step - loss: 0.0361 - mean_sq

#### CNN Lookback 30 

In [16]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)


conv_width = 30
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame30)
model_eval(conv,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 1s 10ms/step - loss: 0.0474 - mean_squared_error: 0.0474 - mean_absolute_error: 0.1051 - val_loss: 0.0563 - val_mean_squared_error: 0.0563 - val_mean_absolute_error: 0.1508
Epoch 2/100
142/142 [==============================] - 1s 6ms/step - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.0905 - val_loss: 0.0469 - val_mean_squared_error: 0.0469 - val_mean_absolute_error: 0.1307
Epoch 3/100
142/142 [==============================] - 1s 6ms/step - loss: 0.0365 - mean_squared_error: 0.0365 - mean_absolute_error: 0.0865 - val_loss: 0.0439 - val_mean_squared_error: 0.0439 - val_mean_absolute_error: 0.1226
Epoch 4/100
142/142 [==============================] - 1s 7ms/step - loss: 0.0354 - mean_squared_error: 0.0354 - mean_absolute_error: 0.0841 - val_loss: 0.0442 - val_mean_squared_error: 0.0442 - val_mean_absolute_error: 0.1195
Epoch 5/100
142/142 [==============================] - 1s 6ms/step - loss: 0.0345 - mean_sq

## BILSTM

### Lookback 1

In [17]:
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame)
model_eval(bilstm,lookback_frame)

Epoch 1/100
144/144 [==============================] - 3s 24ms/step - loss: 0.0452 - mean_squared_error: 0.0452 - mean_absolute_error: 0.1021 - val_loss: 0.0421 - val_mean_squared_error: 0.0421 - val_mean_absolute_error: 0.1212
Epoch 2/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0399 - mean_squared_error: 0.0399 - mean_absolute_error: 0.0930 - val_loss: 0.0417 - val_mean_squared_error: 0.0417 - val_mean_absolute_error: 0.1165
Epoch 3/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0392 - mean_squared_error: 0.0392 - mean_absolute_error: 0.0901 - val_loss: 0.0404 - val_mean_squared_error: 0.0404 - val_mean_absolute_error: 0.1146
Epoch 4/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.0887 - val_loss: 0.0406 - val_mean_squared_error: 0.0406 - val_mean_absolute_error: 0.1117
Epoch 5/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0388 - mean_sq

### BiLSTM Lookback 5

In [18]:
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame5)
model_eval(bilstm,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 5s 33ms/step - loss: 0.0448 - mean_squared_error: 0.0448 - mean_absolute_error: 0.1107 - val_loss: 0.0410 - val_mean_squared_error: 0.0410 - val_mean_absolute_error: 0.1274
Epoch 2/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0390 - mean_squared_error: 0.0390 - mean_absolute_error: 0.0939 - val_loss: 0.0389 - val_mean_squared_error: 0.0389 - val_mean_absolute_error: 0.1185
Epoch 3/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0381 - mean_squared_error: 0.0381 - mean_absolute_error: 0.0897 - val_loss: 0.0390 - val_mean_squared_error: 0.0390 - val_mean_absolute_error: 0.1165
Epoch 4/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0379 - mean_squared_error: 0.0379 - mean_absolute_error: 0.0877 - val_loss: 0.0380 - val_mean_squared_error: 0.0380 - val_mean_absolute_error: 0.1110
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0375 - mean_

Epoch 37/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0362 - mean_squared_error: 0.0362 - mean_absolute_error: 0.0807 - val_loss: 0.0372 - val_mean_squared_error: 0.0372 - val_mean_absolute_error: 0.1038
Epoch 38/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0363 - mean_squared_error: 0.0363 - mean_absolute_error: 0.0808 - val_loss: 0.0373 - val_mean_squared_error: 0.0373 - val_mean_absolute_error: 0.1048
Epoch 39/100
144/144 [==============================] - 2s 13ms/step - loss: 0.0362 - mean_squared_error: 0.0362 - mean_absolute_error: 0.0811 - val_loss: 0.0370 - val_mean_squared_error: 0.0370 - val_mean_absolute_error: 0.1025
Epoch 40/100
144/144 [==============================] - 2s 13ms/step - loss: 0.0361 - mean_squared_error: 0.0361 - mean_absolute_error: 0.0806 - val_loss: 0.0370 - val_mean_squared_error: 0.0370 - val_mean_absolute_error: 0.1026
| setwise accuracy = 0.640432
| eventwise accuracy = 0.910494


### BiLSTM Lookback 15

In [19]:
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame15)
model_eval(bilstm,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 5s 32ms/step - loss: 0.0455 - mean_squared_error: 0.0455 - mean_absolute_error: 0.1129 - val_loss: 0.0422 - val_mean_squared_error: 0.0422 - val_mean_absolute_error: 0.1288
Epoch 2/100
143/143 [==============================] - 2s 13ms/step - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.0942 - val_loss: 0.0410 - val_mean_squared_error: 0.0410 - val_mean_absolute_error: 0.1222
Epoch 3/100
143/143 [==============================] - 3s 19ms/step - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.0894 - val_loss: 0.0396 - val_mean_squared_error: 0.0396 - val_mean_absolute_error: 0.1175
Epoch 4/100
143/143 [==============================] - 2s 11ms/step - loss: 0.0376 - mean_squared_error: 0.0376 - mean_absolute_error: 0.0873 - val_loss: 0.0391 - val_mean_squared_error: 0.0391 - val_mean_absolute_error: 0.1136
Epoch 5/100
143/143 [==============================] - 2s 11ms/step - loss: 0.0373 - mea

Epoch 37/100
143/143 [==============================] - 3s 19ms/step - loss: 0.0358 - mean_squared_error: 0.0358 - mean_absolute_error: 0.0804 - val_loss: 0.0369 - val_mean_squared_error: 0.0369 - val_mean_absolute_error: 0.1022
Epoch 38/100
143/143 [==============================] - 3s 21ms/step - loss: 0.0359 - mean_squared_error: 0.0359 - mean_absolute_error: 0.0801 - val_loss: 0.0371 - val_mean_squared_error: 0.0371 - val_mean_absolute_error: 0.1049
Epoch 39/100
143/143 [==============================] - 2s 17ms/step - loss: 0.0358 - mean_squared_error: 0.0358 - mean_absolute_error: 0.0805 - val_loss: 0.0372 - val_mean_squared_error: 0.0372 - val_mean_absolute_error: 0.1049
Epoch 40/100
143/143 [==============================] - 3s 20ms/step - loss: 0.0358 - mean_squared_error: 0.0358 - mean_absolute_error: 0.0803 - val_loss: 0.0370 - val_mean_squared_error: 0.0370 - val_mean_absolute_error: 0.1028
Epoch 41/100
143/143 [==============================] - 2s 17ms/step - loss: 0.0358 

### BiLSTM Lookback 30

In [20]:
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame30)
model_eval(bilstm,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 8s 57ms/step - loss: 0.0464 - mean_squared_error: 0.0464 - mean_absolute_error: 0.1155 - val_loss: 0.0413 - val_mean_squared_error: 0.0413 - val_mean_absolute_error: 0.1283
Epoch 2/100
142/142 [==============================] - 3s 22ms/step - loss: 0.0392 - mean_squared_error: 0.0392 - mean_absolute_error: 0.0945 - val_loss: 0.0387 - val_mean_squared_error: 0.0387 - val_mean_absolute_error: 0.1164
Epoch 3/100
142/142 [==============================] - 4s 30ms/step - loss: 0.0383 - mean_squared_error: 0.0383 - mean_absolute_error: 0.0899 - val_loss: 0.0378 - val_mean_squared_error: 0.0378 - val_mean_absolute_error: 0.1120
Epoch 4/100
142/142 [==============================] - 4s 30ms/step - loss: 0.0378 - mean_squared_error: 0.0378 - mean_absolute_error: 0.0874 - val_loss: 0.0374 - val_mean_squared_error: 0.0374 - val_mean_absolute_error: 0.1089- loss: 0.0379 - mean_squared_error: 0.0379 - mean_absolute_error: 0. - ETA: 1s - loss: 0

## CLDNN 

### Lookback 1

In [21]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 1
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame)
model_eval(conv_lstm,lookback_frame)

Epoch 1/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0424 - mean_squared_error: 0.0424 - mean_absolute_error: 0.0880 - val_loss: 0.0401 - val_mean_squared_error: 0.0401 - val_mean_absolute_error: 0.1108
Epoch 2/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0393 - mean_squared_error: 0.0393 - mean_absolute_error: 0.0889 - val_loss: 0.0394 - val_mean_squared_error: 0.0394 - val_mean_absolute_error: 0.1093
Epoch 3/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0388 - mean_squared_error: 0.0388 - mean_absolute_error: 0.0868 - val_loss: 0.0389 - val_mean_squared_error: 0.0389 - val_mean_absolute_error: 0.1064
Epoch 4/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0384 - mean_squared_error: 0.0384 - mean_absolute_error: 0.0856 - val_loss: 0.0381 - val_mean_squared_error: 0.0381 - val_mean_absolute_error: 0.1050
Epoch 5/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0381 - mean_sq

### CLDNN Lookback 5

In [22]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 1
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame5)
model_eval(conv_lstm,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 4s 26ms/step - loss: 0.0417 - mean_squared_error: 0.0417 - mean_absolute_error: 0.0896 - val_loss: 0.0426 - val_mean_squared_error: 0.0426 - val_mean_absolute_error: 0.1166
Epoch 2/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0391 - mean_squared_error: 0.0391 - mean_absolute_error: 0.0871 - val_loss: 0.0402 - val_mean_squared_error: 0.0402 - val_mean_absolute_error: 0.1128
Epoch 3/100
144/144 [==============================] - 2s 15ms/step - loss: 0.0387 - mean_squared_error: 0.0387 - mean_absolute_error: 0.0858 - val_loss: 0.0408 - val_mean_squared_error: 0.0408 - val_mean_absolute_error: 0.1110
Epoch 4/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0383 - mean_squared_error: 0.0383 - mean_absolute_error: 0.0851 - val_loss: 0.0396 - val_mean_squared_error: 0.0396 - val_mean_absolute_error: 0.1101
Epoch 5/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0380 - mea

### CLDNN Lookback 15

In [23]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 1
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame15)
model_eval(conv_lstm,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 5s 32ms/step - loss: 0.0415 - mean_squared_error: 0.0415 - mean_absolute_error: 0.0931 - val_loss: 0.0414 - val_mean_squared_error: 0.0414 - val_mean_absolute_error: 0.1148
Epoch 2/100
143/143 [==============================] - 2s 16ms/step - loss: 0.0388 - mean_squared_error: 0.0388 - mean_absolute_error: 0.0870 - val_loss: 0.0391 - val_mean_squared_error: 0.0391 - val_mean_absolute_error: 0.1123
Epoch 3/100
143/143 [==============================] - 3s 19ms/step - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.0855 - val_loss: 0.0393 - val_mean_squared_error: 0.0393 - val_mean_absolute_error: 0.1113
Epoch 4/100
143/143 [==============================] - 3s 23ms/step - loss: 0.0379 - mean_squared_error: 0.0379 - mean_absolute_error: 0.0844 - val_loss: 0.0383 - val_mean_squared_error: 0.0383 - val_mean_absolute_error: 0.1035
Epoch 5/100
143/143 [==============================] - 3s 18ms/step - loss: 0.0377 - mea

### ClDNN Lookback 30

In [24]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 30
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame30)
model_eval(conv_lstm,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 6s 42ms/step - loss: 0.0419 - mean_squared_error: 0.0419 - mean_absolute_error: 0.0905 - val_loss: 0.0393 - val_mean_squared_error: 0.0393 - val_mean_absolute_error: 0.1209
Epoch 2/100
142/142 [==============================] - 2s 12ms/step - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.0871 - val_loss: 0.0376 - val_mean_squared_error: 0.0376 - val_mean_absolute_error: 0.1137
Epoch 3/100
142/142 [==============================] - 2s 12ms/step - loss: 0.0368 - mean_squared_error: 0.0368 - mean_absolute_error: 0.0844 - val_loss: 0.0377 - val_mean_squared_error: 0.0377 - val_mean_absolute_error: 0.1081
Epoch 4/100
142/142 [==============================] - 1s 9ms/step - loss: 0.0355 - mean_squared_error: 0.0355 - mean_absolute_error: 0.0813 - val_loss: 0.0403 - val_mean_squared_error: 0.0403 - val_mean_absolute_error: 0.1150
Epoch 5/100
142/142 [==============================] - 1s 9ms/step - loss: 0.0346 - mean_